Author: Jitender Singh Virk ||
Date created: 25 Dec 2017

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_location = "C:\\Users\\Virksaab\\Desktop\\AI\\datasets_for_code_in_repos\\char74k_dataset\\"

In [ ]:
# Global constants describing the data set.
NUM_CLASSES = 62
IMAGE_SIZE = 64
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE * 3

In [ ]:
# HyperParameters
batch_size = 1
epochs = 2

In [ ]:
# Read tfrecords files in batch
def read_tfrecords_in_batch(filename, batch_size=64, num_epochs=None, shuffle=False):
    # Create a queue
    filename_queue = tf.train.string_input_producer([filename], num_epochs=num_epochs, shuffle=shuffle)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example, features={'image_raw': tf.FixedLenFeature([], tf.string),
                                                                     'label': tf.FixedLenFeature([], tf.int64)})
    # Convert from a scalar string tensor (whose single string has
    # length IMAGE_PIXELS) to a uint8 tensor with shape [IMAGE_PIXELS]
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image.set_shape([IMAGE_PIXELS])
    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)
    # Get data in batch
    image_batch, labels_batch = tf.train.batch([image, label], batch_size=batch_size, num_threads=10, capacity=32)
    return image_batch, labels_batch
    

In [ ]:
filename = data_location+"train.tfrecords"

with tf.Session() as sess:
    
    # Get data in batch(put this line before queue runners and initialization ops)
    image_batch, labels_batch = read_tfrecords_in_batch(filename, batch_size=batch_size, num_epochs=epochs, shuffle=False)
    
    # Op for initializing the variables.
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    
    # Set coordinator and start queue runner
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        batch_no = 0
        while not coord.should_stop():
            batch_no += 1
            # Run training steps or whatever
            data = sess.run([image_batch, labels_batch])
            #print("Batch number:", batch_no)
            if batch_no % 100 == 0:
                img = data[0].reshape(64, 64, 3)
                plt.imshow(img)
                plt.show()
            print(batch_no)
                #print("X:", data[0].shape, " Y:", data[1].shape)


    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')

    finally:
        # When done, ask the threads to stop.
        coord.request_stop()


    # Wait for threads to finish.
    coord.join(threads)
    sess.close() 


Copyright © 2017, Jitender Singh Virk

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.